Given is the boston.csv dataset with the following variable information:
# CRIM - Per Capita crime rate
# ZN - Proportion of residential land zoned for lots over 25000 sq. ft
# INDUS - Proportion of non-retial business acres
# CHAS - Charles River dummy variable (1 - if tracts bounds river, 0 -otherwise)
# NOX - Nitrogen Oxide concentration
# RM - Average number of rooms per dwelling
# AGE - Proportion of owner-occupied unit built prior 1940
# DIS - Weighted MEan of distances of five Boston Employement Centres
# RAD - Index of accessibilities to Radial highways
# TAX - Full-value-property-tax rates per $10,000
# PT - Pupil-teacher Ratio
# B - the proportion of blacks
# LSTAT - Lower Status of the Population (%)
# MV - Median Value of homes (Target Variable)

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName('Questions').getOrCreate()

In [ ]:
spark

# Q.1
# Read the given CSV file in a Hive table 

---



---





# Perform the following tasks using PySpark
# Q2. Read the data from Hive table as spark dataframe

In [ ]:
emp = spark.read.csv('/content/boston.csv',header='true', 
                      inferSchema='true')
emp.show(5)

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|18.0|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|       24.0|
|0.027310001| 0.0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729| 0.0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001| 0.0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999| 0.0|2.

# Q3. Get the correlation between dependent and independent variables

In [ ]:
import six
for i in emp.columns:
    if not( isinstance(emp.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to MV for ", i, emp.stat.corr('MV',i))

Correlation to MV for  CRIM -0.3883046116575088
Correlation to MV for  ZN 0.36044534463752903
Correlation to MV for  INDUS -0.48372517128143383
Correlation to MV for  CHAS 0.17526017775291847
Correlation to MV for  NOX -0.4273207763683772
Correlation to MV for  RM 0.695359937127267
Correlation to MV for  AGE -0.37695456714288667
Correlation to MV for  DIS 0.24992873873512172
Correlation to MV for  RAD -0.3816262315669168
Correlation to MV for  TAX -0.46853593528654536
Correlation to MV for  PT -0.5077867038116085
Correlation to MV for  B 0.3334608226834164
Correlation to MV for  LSTAT -0.7376627294671615
Correlation to MV for  MV 1.0


In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')


In [ ]:
vhouse_df = vectorAssembler.transform(emp)


In [ ]:
vhouse_df =vhouse_df.select(['features', 'MV'])
vhouse_df.show(5)

+--------------------+-----------+
|            features|         MV|
+--------------------+-----------+
|[0.00632,18.0,2.3...|       24.0|
|[0.027310001,0.0,...|21.60000038|
|[0.02729,0.0,7.07...|34.70000076|
|[0.032370001,0.0,...|33.40000153|
|[0.069049999,0.0,...|36.20000076|
+--------------------+-----------+
only showing top 5 rows



# Q4. Build a linear regression model to predict house price

In [ ]:
#splitting the data for training and testing

splits = vhouse_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [ ]:
#linear regression model

from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-0.053824402967175985,0.013208516088527572,-0.0024346709286272826,1.4652621136360078,-6.521749786768949,4.145981901490713,0.0,-0.622118616943058,0.0,0.0,-0.7549884405758134,0.00830008172144777,-0.48421657714838023]
Intercept: 19.584768309879994


# Q5. Evaluate the Linear Regression model by getting the RMSE and R-squared values

In [ ]:
#accuracy measure for model

trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 4.918640
r2: 0.711394


In [ ]:
train_df.describe().show()

+-------+-----------------+
|summary|               MV|
+-------+-----------------+
|  count|              364|
|   mean|22.42857144432143|
| stddev|9.168319343339927|
|    min|              5.0|
|    max|             50.0|
+-------+-----------------+



In [ ]:
#evaluating

lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","MV","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="MV",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
| 31.71840352783358|32.70000076|[0.01301,35.0,1.5...|
|37.985225568826365|       50.0|[0.01381,80.0,0.4...|
|40.292147941197044|       50.0|[0.01501,90.0,1.2...|
|31.227189596737396|32.90000153|[0.01778,95.0,1.4...|
| 26.26708992378174|23.10000038|[0.0187,85.0,4.15...|
+------------------+-----------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.714176


In [ ]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 4.95326


In [ ]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

numIterations: 10
objectiveHistory: [0.5, 0.42996112792008406, 0.23644653973608518, 0.21603069807285244, 0.18507019330055458, 0.18250030869343067, 0.182053501174409, 0.18140479419321673, 0.18045916491675626, 0.18022882424344305, 0.18016206507216415]
+-------------------+
|          residuals|
+-------------------+
| -6.354801541412655|
| 1.2783881301975057|
| -5.716004953995466|
|  5.171624702499564|
| 0.9535201298288847|
|0.35068362099526595|
|-1.3928372038384502|
|-3.1259933372025444|
|  9.147225089622744|
|  3.478904217861359|
|  6.863679663320376|
|-1.0337370052690602|
| 11.008188739560268|
|  6.010486577234467|
|  -9.96043276305442|
|-3.9694180823404714|
| 1.6662880037378685|
|-1.9022149807708608|
| 0.4796073552686373|
|  2.591677359414728|
+-------------------+
only showing top 20 rows



In [ ]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","MV","features").show()

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
| 31.71840352783358|32.70000076|[0.01301,35.0,1.5...|
|37.985225568826365|       50.0|[0.01381,80.0,0.4...|
|40.292147941197044|       50.0|[0.01501,90.0,1.2...|
|31.227189596737396|32.90000153|[0.01778,95.0,1.4...|
| 26.26708992378174|23.10000038|[0.0187,85.0,4.15...|
|26.217765298964704|24.70000076|[0.02055,85.0,0.7...|
|31.128669239182543|31.10000038|[0.02187,60.0,2.9...|
|31.076474259855726|34.70000076|[0.02729,0.0,7.07...|
|25.468377883047694|21.60000038|[0.027310001,0.0,...|
| 20.54657716980188|       17.5|[0.031129999,0.0,...|
| 29.56651154135823|33.40000153|[0.032370001,0.0,...|
|22.254941274448804|20.60000038|[0.033059999,0.0,...|
|20.588202376713625|       19.5|[0.03427,0.0,5.19...|
| 28.18026971827286|24.10000038|[0.034449998,82.5...|
|  24.5036713202785|19.39999962|[0.03466,35.0,6.0...|
|27.628127424477256|       2